In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [20]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model= "models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model= "gemini-1.0-pro")

In [3]:
# from langchain_huggingface import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name= "all-MiniLM-L6-v2")
# from langchain_groq import ChatGroq
# llm = ChatGroq(model_name= "Gemma2-9b-It")

# Simple AI Assistant

In [4]:
while True:
    question = input("Type your question. If you want to quit the chat write quit.")
    if question != "quit":
        print(llm.invoke(question).content)
    else:
        print("Good Bye Take Care")
        break

Good Bye Take Care


In [5]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [6]:
store = {} 

In [7]:
def get_session_histroy(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [8]:
config = {"configurable": {"session_id": "FirstChat"}}

In [9]:
model_with_memmory = RunnableWithMessageHistory(llm, get_session_histroy)

In [10]:
model_with_memmory.invoke(("Hi, my name is Dhruv Saxena"), config= config).content

'Hello Dhruv Saxena! How can I assist you today?'

In [11]:
model_with_memmory.invoke(("tell me my name?"), config= config).content

'Your name is Dhruv Saxena.'

In [12]:
store

{'FirstChat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, my name is Dhruv Saxena', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Dhruv Saxena! How can I assist you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-8afb7a87-9eba-4dbe-94d1-f461f826298c-0', usage_metadata={'input_tokens': 11, 'output_tokens': 14, 'total_tokens': 25}), HumanMessage(content='tell me my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Dhruv Saxena.', a

## RAG With LCEL

In [13]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [16]:
### Reading the txt files from source directory

loader = DirectoryLoader("data", glob= "./*.txt", loader_cls= TextLoader)
docs = loader.load()

## Creating Chunks unsing RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 150,
    chunk_overlap= 10,
    length_function= len 
) 

new_docs = text_splitter.split_documents(documents= docs)
doc_strings = [doc.page_content for doc in new_docs]


In [17]:
new_docs

[Document(metadata={'source': 'data\\llama3.txt'}, page_content='Modern artificial intelligence (AI) systems are powered by foundation models. This paper presents a'),
 Document(metadata={'source': 'data\\llama3.txt'}, page_content='new set of foundation models, called Llama 3. It is a herd of language models that natively support'),
 Document(metadata={'source': 'data\\llama3.txt'}, page_content='multilinguality, coding, reasoning, and tool usage. Our largest model is a dense Transformer with'),
 Document(metadata={'source': 'data\\llama3.txt'}, page_content='405B parameters and a context window of up to 128K tokens. This paper presents an extensive'),
 Document(metadata={'source': 'data\\llama3.txt'}, page_content='empirical evaluation of Llama 3. We find that Llama 3 delivers comparable quality to leading language'),
 Document(metadata={'source': 'data\\llama3.txt'}, page_content='models such as GPT-4 on a plethora of tasks. We publicly release Llama 3, including pre-trained and'),


In [19]:
### BGE Embeddings

# from langchain.embeddings import HuggingFaceBgeEmbeddings

# model_name = "BAAI/bge-base-en-v1.5"
# model_kwargs = {"device": "cpu"}
# encode_kwargs = {"normalize_embeddings": True}
# embeddings = HuggingFaceBgeEmbeddings(
#     model_name= model_name,
#     model_kwargs= model_kwargs,
#     encode_kwargs= encode_kwargs
# )

In [21]:
### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 3})

In [22]:
template = """Answer the question based on the following context:
{context}

Question: {question}

Answer: 
"""

prompt = PromptTemplate.from_template(template)

In [23]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

In [24]:
question = "Waht is llama3? Can you highlight 3 important points?"
print(retrieval_chain.invoke(question))

1. Llama 3 is a multimodal model.
2. Llama 3 is safe, as detailed in Section 5.4.
3. Llama 3 has extensions that enable it to perform a variety of tasks.


# Let's Start with Tools and Agents

In [26]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [28]:
api_wrapper = WikipediaAPIWrapper()

In [29]:
tool= WikipediaQueryRun(api_wrapper= api_wrapper)

In [30]:
tool.name

'wikipedia'

In [31]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [32]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Retrieval-augmented generation
Summary: Retrieval augmented generation (RAG) is a type of generative artificial intelligence that has information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information in preference to information drawn from its own vast, static training data. This allows LLMs to use domain-specific and/or updated information.  
Use cases include providing chatbot access to internal company data, or giving factual information only from an authoritative source.



Page: DataStax
Sum

In [33]:
from langchain_community.tools import YouTubeSearchTool

In [34]:
YTtool= YouTubeSearchTool()

In [35]:
YTtool.name

'youtube_search'

In [36]:
YTtool.description

'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional'

In [40]:
YTtool.run("sunny savita") 

"['https://www.youtube.com/watch?v=8aUYzb1aYDU&pp=ygUMc3Vubnkgc2F2aXRh', 'https://www.youtube.com/watch?v=ABK00e2XdPo&pp=ygUMc3Vubnkgc2F2aXRh']"

In [41]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [42]:
tool = TavilySearchResults()

In [43]:
tool.name

'tavily_search_results_json'

In [44]:
tool.description

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

In [45]:
tool.invoke({"query": "Waht happened in the latest burning man floods"})

[{'url': 'https://www.npr.org/2023/09/03/1197497458/the-latest-on-the-burning-man-flooding',
  'content': "There are also reports that at least one person has died at the counterculture festival about a hundred miles north of Reno, Nev. Earlier this afternoon, I caught up with NPR's Claudia Peschiutta, who's at her first burn, and she told me it's muddy where she is, but that she and her camp family have been making the best of things.\n National\nThe latest on the Burning Man flooding\nClaudia Peschiutta\nAuthorities are investigating a death at the Burning Man festival in the Nevada desert after tens of thousands of people are stuck in camps because of rain.\n SCOTT DETROW, HOST:\nKnee-deep mud, warnings to conserve food and water, orders to shelter in place - this is all at Burning Man 2023 after torrential rains turned the Black Rock Desert into miles and miles of mud. I mean, mostly what I've seen from my personal experience is just any sort of need that you have, somebody, whethe

In [46]:
from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [52]:
tool = load_tools(["wikipedia"], llm= llm)

In [53]:
agent= initialize_agent(tool, llm, agent= AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [54]:
agent.run("What is current GDP of India?")



> Entering new AgentExecutor chain...
Action: wikipedia
Action Input: What is current GDP of India
Observation: Page: List of countries by GDP (nominal)
Summary: Gross domestic product (GDP) is the market value of all final goods and services from a nation in a given year. Countries are sorted by nominal GDP estimates from financial and statistical institutions, which are calculated at market or government official exchange rates. Nominal GDP does not take into account differences in the cost of living in different countries, and the results can vary greatly from one year to another based on fluctuations in the exchange rates of the country's currency. Such fluctuations may change a country's ranking from one year to the next, even though they often make little or no difference in the standard of living of its population.
Comparisons of national wealth are also frequently made based on purchasing power parity (PPP), to adjust for differences in the cost of living in different countri

"I'm sorry, but I cannot provide an exact value for the current GDP of India. However, the latest data I have available from Wikipedia shows that India's nominal GDP was estimated to be around $3.5 trillion in 2022. It is important to note that GDP figures can change frequently, and the most up-to-date information may be available on the websites of relevant financial institutions."